<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/Rizwan_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
!pip install caas_jupyter_tools

ERROR: Could not find a version that satisfies the requirement caas_jupyter_tools (from versions: none)
ERROR: No matching distribution found for caas_jupyter_tools


In [8]:
# # PAAm Hydrogel One-to-Many Inverse Design (Colab-ready)
# This single cell:
# 1) Loads your **stress–strain datasets** (from the uploaded Excel file)
# 2) Normalizes units (MPa→kPa; % or %×100 → fraction)
# 3) Derives properties (E, UTS, fracture strain, toughness)
# 4) Builds a unified **design library**
# 5) Performs **one-to-many inverse design** (k-NN retrieval)
# 6) Suggests **NCC/TEMPO** loading% for a target UTS
#
# 👉 Just run this cell after uploading the `paam_hydrogel_stress_strain_data_v2.xlsx` file.

import pandas as pd
import numpy as np
import re
import os

# Path to the uploaded file (update this if necessary)
file_path = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# Load Excel data
xls = pd.ExcelFile(file_path)

# Function to normalize strain and stress data
def _normalize_strain(e, is_percent):
    e = np.asarray(e, dtype=float)
    if not is_percent:
        return e  # already fractional strain (or mm for raw Excel; see note where handled)
    emax = np.nanmax(e) if np.isfinite(e).any() else 0.0
    if emax > 100:  # stored as %×100 (e.g., 1800 for 18%)
        return e / 10000.0
    return e / 100.0  # standard percent 0..100

def _normalize_stress(s, stress_col):
    s = np.asarray(s, dtype=float)
    name = str(stress_col).strip().lower()
    if "mpa" in name:
        return s * 1000.0  # MPa -> kPa
    return s

def _compute_properties(strain, stress, strain_is_percent=False, strain_col="", stress_col=""):
    s_raw = np.asarray(stress, dtype=float)
    e_raw = np.asarray(strain, dtype=float)
    s = _normalize_stress(s_raw, stress_col)
    e = _normalize_strain(e_raw, strain_is_percent) # Removed strain_col here

    mask = np.isfinite(e) & np.isfinite(s)
    e, s = e[mask], s[mask]
    if len(e) < 5:
        return None

    idx = np.argsort(e)
    e, s = e[idx], s[idx]

    # Linear region (up to 10% strain or first 15% span)
    e_max, e_min = float(np.nanmax(e)), float(np.nanmin(e))
    e_thresh = min(0.10, e_min + 0.15*(e_max - e_min))
    lin_mask = e <= e_thresh
    if lin_mask.sum() < 5:
        k = min(10, len(e))
        lin_mask = np.zeros_like(e, dtype=bool)
        lin_mask[:k] = True

    e_lin, s_lin = e[lin_mask], s[lin_mask]
    try:
        slope, intercept = np.polyfit(e_lin, s_lin, 1)
    except Exception:
        slope, intercept = np.nan, np.nan

    uts = float(np.nanmax(s))
    frac_strain = float(np.nanmax(e))
    area = float(np.trapz(s, e))  # kPa·strain
    return {
        "E_initial_kPa": float(slope),
        "UTS_kPa": uts,
        "Fracture_Strain": frac_strain,
        "Toughness_kPa": area,
        "n_points": int(len(e)),
    }

# Parse the Excel sheets into the unified design library
def _tidy_wide(df, source_name):
    cols = list(df.columns)
    pairs = []
    for c in cols:
        c_norm = str(c).strip().lower()
        if "strain" in c_norm:
            m = re.search(r"_(.+)$", str(c))
            key = m.group(1) if m else re.sub(r"[^A-Za-z0-9]+", "_", c_norm)
            stress_candidates = [col for col in cols
                                 if str(col).strip().lower().startswith("stress")
                                 and str(col).endswith(f"_{key}")]
            if stress_candidates:
                is_pct = ("% )" in str(c)) or ("(%)" in str(c)) or ("%)" in str(c))
                pairs.append((key, c, stress_candidates[0], is_pct))
    records = []
    for key, strain_col, stress_col, is_pct in pairs:
        props = _compute_properties(df[strain_col].values, df[stress_col].values,
                                    strain_is_percent=is_pct,
                                    strain_col=strain_col, stress_col=stress_col)
        if props is None:
            continue
        row = {
            "source": source_name,
            "label": key,
            "family": key.split("_")[0],
            "loading_pct": np.nan,
            "notes": f"Derived from {strain_col} vs {stress_col} (units normalized).",
        }
        row.update(props)
        records.append(row)
    return pd.DataFrame.from_records(records)

# Load each sheet and extract data
wide_features = []
for sheet in xls.sheet_names:
    df = xls.parse(sheet)
    wide_features.append(_tidy_wide(df, source_name=sheet))
wide_features = pd.concat(wide_features, ignore_index=True)

# --------------------------------------------------------------------------------------
# Inverse design: k-NN in property space
# --------------------------------------------------------------------------------------
def inverse_design_search(library: pd.DataFrame, targets: dict, weights: dict=None, k: int=6) -> pd.DataFrame:
    props = [p for p in targets.keys() if p in library.columns]
    df = library.dropna(subset=props).copy()
    if df.empty:
        return df
    w = {p: 1.0 for p in props}
    if weights:
        w.update(weights)
    for p in props:
        df[f"err_{p}"] = (df[p] - targets[p])**2 * (w[p]**2)
    df["score"] = np.sqrt(df[[f"err_{p}" for p in props]].sum(axis=1))
    cols = ["source","label","family","loading_pct"] + props + ["score"]
    return df.sort_values("score").head(k)[cols]

# Example demo search (you can modify these)
goals = [
    ("High strength, moderate ductility", {"UTS_kPa": 1500.0, "Fracture_Strain": 0.50}),
    ("High toughness", {"Toughness_kPa": 800.0}),
    ("Stiff, low strain", {"E_initial_kPa": 3000.0, "Fracture_Strain": 0.20}),
]

# Run inverse design search for each goal and collect results
all_results = []
for title, tgt in goals:
    res = inverse_design_search(wide_features, tgt, k=6).copy()
    res.insert(0, "Objective", title)
    all_results.append(res)

inv_results = pd.concat(all_results, ignore_index=True)

# Show results in Colab (will appear as a table)
# import caas_jupyter_tools
# caas_jupyter_tools.display_dataframe_to_user("Inverse Design Results", inv_results)
display(inv_results)

# Optionally, save results for later
inv_results.to_csv("/content/inverse_design_demo_results.csv", index=False)
print("Results saved to 'inverse_design_demo_results.csv'")

/tmp/ipython-input-2666979128.py:71: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = float(np.trapz(s, e))  # kPa·strain
/tmp/ipython-input-2666979128.py:71: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = float(np.trapz(s, e))  # kPa·strain
/tmp/ipython-input-2666979128.py:71: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = float(np.trapz(s, e))  # kPa·strain
/tmp/ipython-input-2666979128.py:71: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = float(np.trapz(s, e))  # kPa·strain


,Objective,source,label,family,loading_pct,UTS_kPa,Fracture_Strain,score,Toughness_kPa,E_initial_kPa
0,"High strength, moderate ductility",Stress-Strain_Data__Panels_a___,G3-45,G3-45,NaN,1700.0,0.80,200.000225,NaN,NaN
1,"High strength, moderate ductility",Stress-Strain_Data__Panels_a___,G1.5-45,G1.5-45,NaN,1800.0,0.80,300.000150,NaN,NaN
2,"High strength, moderate ductility",Stress-Strain_Data__Panels_a___,G2-30,G2-30,NaN,1100.0,0.40,400.000012,NaN,NaN
3,"High strength, moderate ductility",Stress-Strain_Data__Panels_a___,G1-45,G1-45,NaN,1000.0,0.80,500.000090,NaN,NaN
4,"High strength, moderate ductility",Stress-Strain_Data__Panels_a___,G2.5-45,G2.5-45,NaN,2100.0,0.80,600.000075,NaN,NaN
5,"High strength, moderate ductility",Stress-Strain_Data__Panels_a___,G2-40,G2-40,NaN,2100.0,0.80,600.000075,NaN,NaN
6,High toughness,Stress-Strain_Data__Panels_a___,G3-45,G3-45,NaN,NaN,NaN,90.000000,710.0,NaN
7,High toughness,Stress-Strain_Data__Panels_a___,G2-40,G2-40,NaN,NaN,NaN,110.000000,910.0,NaN
8,High toughness,Stress-Strain_Data__Panels_a___,G2.5-45,G2.5-45,NaN,NaN,NaN,110.000000,910.0,NaN
9,High toughness,Stress-Strain_Data__Panels_a___,G1.5-45,G1.5-45,NaN,NaN,NaN,140.000000,660.0,NaN


Results saved to 'inverse_design_demo_results.csv'
